#### import libraries 

In [2]:
import cv2
import pandas as pd
import numpy as np
from collections import Counter

#### read image(s)

In [60]:
img = cv2.imread("./data/masks/0363100012175996_4.372222.png")
cv2.imshow("Display window", img)
k = cv2.waitKey(0) # Wait for a keystroke in the window

### Extract only the windows from the mask

In [61]:
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert the image to HSV for better color segmentation
hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# save CSV to read the color values of the image 
hsv_2d = hsv.reshape((-1,3))
df = pd.DataFrame(hsv_2d)
tiles_csv_path = './data/junk/hsv.csv'
df.to_csv(tiles_csv_path, index=False, header=False)

'not integrated in the code, just read from the file'

# retrieved color values
sky_val = np.array([0,0,255])
cl_fac_val = np.array([60,185,160])
window_fac_val = np.array([102,211,180])
door_fac_val = np.array([14,241,255])

mask = cv2.inRange(hsv, window_fac_val, window_fac_val)

#save image
cv2.imwrite("data/cv2_window_mask.png", mask)
img_shape= img.shape
print(img_shape)

# show image 
# cv2.imshow("img", mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

(900, 151, 3)


### simplify windows to xy minimum bounding rectangles 

In [65]:
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Approximate contours to polygons + get bounding rects
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)

for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
print(boundRect)                                #  in order of X, Y,(top-left coordinate of the rectangle) and width, height 


drawing = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
# Draw polygonal contour + bonding rects + circles
for i in range(len(contours)):
    color = (0,0,255)
    cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), 
    (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
cv2.imwrite("data/cv2_window_contours.png", drawing)
cv2.imshow('Contours', drawing)
k = cv2.waitKey(0) # Wait for a keystroke in the window

[(21, 839, 47, 38), (15, 743, 30, 70), (60, 742, 30, 70), (104, 739, 31, 69), (14, 667, 30, 49), (61, 665, 30, 51), (107, 659, 30, 58), (13, 598, 31, 37), (106, 594, 31, 39), (61, 594, 30, 47), (61, 534, 31, 31)]


### retrieve additional window information

In [66]:
#list heights of the upper and lower corner of each window
y_bounds = []
window_area = []
window_height = []
for border in boundRect:
    y_bound = []
    y_bound.append(border[1])
    upper_corner = border[1] + border[3]
    y_bound.append(upper_corner)
    y_bounds.append(y_bound)
    window_area.append(border[2]*border[3])
    window_height.append(border[3])
print(y_bounds)
print(window_area)
print(window_height)

[[839, 877], [743, 813], [742, 812], [739, 808], [667, 716], [665, 716], [659, 717], [598, 635], [594, 633], [594, 641], [534, 565]]
[1786, 2100, 2100, 2139, 1470, 1530, 1740, 1147, 1209, 1410, 961]
[38, 70, 70, 69, 49, 51, 58, 37, 39, 47, 31]


### define window clusters

In [68]:
def find_window_clusters(window_y_bounds): 
    window_clusters = []
    for index, window_1 in enumerate(window_y_bounds):
        new_window = [
            min(window_1[0], min(window_2[0] for window_2 in y_bounds if window_2[0] <= window_1[1] and window_1[0] <= window_2[1])),
            max(window_1[1], max(window_2[1] for window_2 in y_bounds if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]))
        ]
        window_clusters.append(new_window)
    return window_clusters

window_clusters = find_window_clusters(y_bounds)
print(window_clusters)

[[839, 877], [739, 813], [739, 813], [739, 813], [659, 717], [659, 717], [659, 717], [594, 641], [594, 641], [594, 641], [534, 565]]


### calculate relevant window- & floor information 

In [69]:
def get_cluster_info(values, boundaries, heights, img_shape):
    counts = Counter(tuple(sublist) for sublist in boundaries)
    duplicate_indices = {k: [i for i, sublist in enumerate(boundaries) if tuple(sublist) == k] 
                         for k, _ in counts.items()}
    
    cluster_info = []
    previous_floor_level = None
    for _, original_boundary in duplicate_indices.items():
        area_sum = sum(values[i] for i in original_boundary)
        avg_height = sum(heights[i] for i in original_boundary) / len(original_boundary)
        middle_boundary = (boundaries[original_boundary[0]][0] + boundaries[original_boundary[0]][1]) / 2
        avg_sill_height = middle_boundary + avg_height / 2
        floor_level = img_shape[0] - (avg_sill_height + 27) #29px is an assumption of a consistent .9m sill height
        
        if previous_floor_level is not None:
            floor_height = floor_level - previous_floor_level
        else:
            floor_height = floor_level  
        wall_area = floor_height * img_shape[1]
        WWR = area_sum/wall_area

        cluster_info.append({
            "count": len(original_boundary),
            "indices": original_boundary,
            "boundary": boundaries[original_boundary[0]],
            "window_area_sum": area_sum,
            "avg_height": avg_height,
            # "floor_level": floor_level,
            "floor_height": floor_height,
            "avg_sill_height": avg_sill_height,
            "WWR": WWR
        })
        previous_floor_level = floor_level  # Update previous_floor_level HERE

    return cluster_info
cluster_info = get_cluster_info(window_area, window_clusters, window_height, img_shape)

for cluster in cluster_info: 
    print(cluster)

{'count': 1, 'indices': [0], 'boundary': [839, 877], 'window_area_sum': 1786, 'avg_height': 38.0, 'floor_height': -4.0, 'avg_sill_height': 877.0, 'WWR': -2.956953642384106}
{'count': 3, 'indices': [1, 2, 3], 'boundary': [739, 813], 'window_area_sum': 6339, 'avg_height': 69.66666666666667, 'floor_height': 66.16666666666663, 'avg_sill_height': 810.8333333333334, 'WWR': 0.6344604400553826}
{'count': 3, 'indices': [4, 5, 6], 'boundary': [659, 717], 'window_area_sum': 4740, 'avg_height': 52.666666666666664, 'floor_height': 96.5, 'avg_sill_height': 714.3333333333334, 'WWR': 0.32529252307586726}
{'count': 3, 'indices': [7, 8, 9], 'boundary': [594, 641], 'window_area_sum': 3766, 'avg_height': 41.0, 'floor_height': 76.33333333333337, 'avg_sill_height': 638.0, 'WWR': 0.3267300963012231}
{'count': 1, 'indices': [10], 'boundary': [534, 565], 'window_area_sum': 961, 'avg_height': 31.0, 'floor_height': 73.0, 'avg_sill_height': 565.0, 'WWR': 0.08718134809035653}


WHOEEEEEEE yay I think I retrieved all needed information for a facade! 

Keep into mind!!! cluster info is a list of dicts now, very annoying

### my thoughts

In [ ]:
# test, grouping windows 
window_clusters = []
for index, window_1 in enumerate(y_bounds): 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds: 
        if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
            cluster.append(window_2)
    window_clusters.append(cluster)  

print(window_clusters)
print(y_bounds)

[0, [[738, 811], [735, 812], [733, 761]], 1, [[738, 811], [735, 812], [733, 761]], 2, [[738, 811], [735, 812], [733, 761]], 3, [[661, 719], [660, 718], [658, 720]], 4, [[661, 719], [660, 718], [658, 720]], 5, [[661, 719], [660, 718], [658, 720]], 6, [[588, 636], [586, 634], [584, 636]], 7, [[588, 636], [586, 634], [584, 636]], 8, [[588, 636], [586, 634], [584, 636]], 9, [[526, 568]]]
[[738, 811], [735, 812], [733, 761], [661, 719], [660, 718], [658, 720], [588, 636], [586, 634], [584, 636], [526, 568]]


In [43]:
last_floor_h = []
# for the lowest average sill height, take difference with 900

for cluster in cluster_info: 
    floor_height = (cluster['floor_height'])
    wall_area = floor_height * img
    WWR = cluster["window_area_sum"]/wall_area
    cluster["WWR"] = WWR
    # for checking if legitimate, devide by 30 to transpose pixels to meters
    print(cluster)


{'count': 3, 'indices': [0, 1, 2], 'boundary': [733, 812], 'window_area_sum': 5689, 'avg_height': 59.333333333333336, 'floor_height': 70.83333333333337, 'avg_sill_height': 802.1666666666666, 'WWR': 0.5283900928792568}
{'count': 3, 'indices': [3, 4, 5], 'boundary': [658, 720], 'window_area_sum': 5994, 'avg_height': 59.333333333333336, 'floor_height': 83.5, 'avg_sill_height': 718.6666666666666, 'WWR': 0.47226599432713523}
{'count': 3, 'indices': [6, 7, 8], 'boundary': [584, 636], 'window_area_sum': 4984, 'avg_height': 49.333333333333336, 'floor_height': 84.0, 'avg_sill_height': 634.6666666666666, 'WWR': 0.39035087719298245}
{'count': 1, 'indices': [9], 'boundary': [526, 568], 'window_area_sum': 1302, 'avg_height': 42.0, 'floor_height': 66.66666666666663, 'avg_sill_height': 568.0, 'WWR': 0.12848684210526323}


In [44]:
# window_clusters = []
# for index, window_1 in enumerate(y_bounds):
#     for window_2 in y_bounds:
#         if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
#             if window_2[0] < window_1[0]:
#                 window_1[0] = window_2[0] 
#             else: 
#                 window_1[0] = window_1[0]
#             if window_2[1] > window_1[1]: 
#                 window_1[1] = window_2[1]
#             else:
#                 window_1[1] = window_1[1] 
#     window_clusters.append(window_1)
# print(window_clusters)
# # count = Counter(tuple(x) for x in window_clusters)
# # print(count)


In [45]:
# def count_duplicates_with_indices(list_of_lists):
#     counts = Counter(tuple(sublist) for sublist in list_of_lists)
#     duplicates = {k: [i for i, sublist in enumerate(list_of_lists) if tuple(sublist) == k] 
#                   for k, v in counts.items()}
#     return duplicates

# # Example usage (assuming 'y_bounds' is your list of lists):
# duplicate_indices = count_duplicates_with_indices(window_clusters)
# print(duplicate_indices)

# def sum_values_by_duplicate_indices(values, duplicate_indices):
#     sums = {}
#     for duplicate_list in duplicate_indices.values():
#         total = 0
#         for index in duplicate_list:
#             total += values[index]
#         sums[tuple(duplicate_list)] = total
#     return sums

# # Example usage (assuming 'window_area' and 'duplicate_indices' are defined):
# area_sums = sum_values_by_duplicate_indices(window_area, duplicate_indices)
# print(window_area)
# print(area_sums)

# def get_duplicate_info_with_boundaries(boundaries, duplicate_indices):
#     result = []
#     for duplicate_list, area_sum in sum_values_by_duplicate_indices(window_area, duplicate_indices).items():
#         original_boundary = boundaries[duplicate_list[0]]  # Take the first boundary as they are similar
#         result.append({
#             "count": len(duplicate_list),
#             "indices": list(duplicate_list),
#             "boundary": original_boundary,  # Use 'boundary' instead of 'boundaries'
#             "area_sum": area_sum
#         })
#     return result

# # Example usage (assuming 'window_clusters', 'window_area', and 'duplicate_indices' are defined):
# duplicate_info = get_duplicate_info_with_boundaries(window_clusters, duplicate_indices)
# print(duplicate_info)

In [46]:
# # test, editing the upper and lower margin of each group based on the extremes that are collected
# # trying to do this without the step of first making the sub-lists per window, but instantly adapting the values when finding a next window
# window_clusters = []
# for index, window_1 in enumerate(y_bounds):
#     # low_bound = []
#     # high_bound = []
#     # window_clusters.append(index)
#     # cluster = []
#     for window_2 in y_bounds:
#         if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
#             if window_2[0] < window_1[0]:
#                 window_1[0] = window_2[0] 
#             else: 
#                 window_1[0] = window_1[0]
#             if window_2[1] > window_1[1]: 
#                 window_1[1] = window_2[1]
#                 # high_bound = window_2[1]
#             else:
#                 window_1[1] = window_1[1] 
#                 # high_bound = window_1[1]
#         # print(window_2)
#     # print(window_1) 

#     # print(low_bound)
#     # print(high_bound)
        
#         # high_bound = window_1[1] # high bound is still wrong, trying to wrap my head around it :) 
 
#         # else:
#             # pass
#     # print(low_bound)
#     # print(high_bound)
#             # high_bound = window_2[1]
#             # print(high_bound)
#         #         new_boundary.append(window_2[1])
#         #     else: 
#         #         new_boundary.append(window_1[0])
#         #     print(new_boundary)       
#             # print("yay!")
#             # cluster.append(window_2)
#         # else: 
#             # print("nay..")
#         # if window_1 is in cluster: 
#     window_clusters.append(window_1)
#     # window_clusters.append(low_bound) 
# print(window_clusters)
# # print(window_clusters) 
# # tuple(x) for x in window_clusters:
# #     my_dict = {i:window_clusters.count(i) for i in window_clusters}
# # print(my_dict)
# count = Counter(tuple(x) for x in window_clusters)
# print(count)
# # >>> Counter(tuple(x) for x in lis)
# # Counter({(12, 34, 56): 3, (45, 78, 334): 1, (56, 90, 78): 2})

In [47]:
# for index, window_1 in enumerate(y_bounds):
#     print(window_1) 
#     window_clusters.append(index)
#     cluster = []
#     for window_2 in y_bounds:
#         high_bound = []
#         print(window_1[0], window_2[1])
#         if window_1[0] <= window_2[1]:
        
#             high_bound = window_2[1]
#             print(high_bound)

todo's